## **Tech Challenge**

### **Projeto**

Entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença, utilizando o estudo do PNAD-COVID 19 do IBGE para termos respostas ao problema proposto, pois são dados confiáveis, porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas, mas há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

• Características clínicas dos sintomas;  
• Características da população;  
• Características econômicas da sociedade.  

Dessa forma, acessar os dados do PNAD-COVID-19 do IBGE (https://covid19.ibge.gov.br/pnad-covid/) e organizar esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

a. Utilização de no máximo 20 questionamentos realizados na pesquisa;  
b. Utilizar 3 meses para construção da solução;  
c. Caracterização dos sintomas clínicos da população;  
d. Comportamento da população na época da COVID-19;  
e. Características econômicas da Sociedade;  

Com objetivo de trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19

### Arquitetura de Dados

![Arquitetura](https://github.com/RicardViana/fiap-Big-Data/blob/main/arquitetura.jpeg?raw=true)

1) Dados utilizado referente a Pesquisa Nacional por Amostra de Domicílios (PNAD COVID19) através do link [PNAD](https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=microdados&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19) *
2) Devido a volumetria dos dados, é feito o armazenamento no AWS S3 e usando camandas medalhão
3) Realizar o carregamento dos da camanda GOLD para o banco de dados PostgreSQL criado via AWS RDS
4) Construir a analise utilizando o Power BI conectado ao Data Mart

\* Os dados foram armazenados no GitHub para a construção desse trabalho

### **Observações**

#### **VS Code**

Para correto funcionamento, sempre reiniciar o Kernel antes de rodar o código

#### **Bibliotecas**

Para correto funcionamento, caso não tenha instalado as seguintes bibliotes: 

- `requests`
- `pandas`  
- `boto3`  
- `python-dotenv`  
- `SQLAlchemy`  
- `psycopg2-binary`  

Realizar a instalação via PIP INSTALL (https://pypi.org/) ou CONDA-FORGE (https://anaconda.org/conda-forge)

#### **Conexões**

Para realizar as conexões é utilizado um arquivo .env salvo no mesmo local do projeto e com a seguinte estrutura

```ini
# Credenciais do PostgreSQL PNAD
POSTGRES_USER_PNAD=
POSTGRES_PASSWORD_PNAD=
POSTGRES_HOST_PNAD=
POSTGRES_PORT_PNAD=5432
POSTGRES_DB_PNAD=

# Credenciais AWS
aws_access_key_id=
aws_secret_access_key=
aws_session_token=

region=us-east-1
output=json
```

#### **Configuração do PostgreSQL na AWS RDS**

##### 1. **Criar instância RDS com PostgreSQL (SandBox)**

1. Acesse o console AWS → [https://us-east-1.console.aws.amazon.com/rds/home?region=us-east-1#](https://console.aws.amazon.com/rds/)
2. Clique em **Criar banco de dados**
3. Selecione:
   - **Tipo de banco:** PostgreSQL
   - **Versão:** PostgreSQL 15 (ou mais recente)
   - **Modelo de uso:** SandBox
   - **Identificador da instância:** `postgres-tc`
   - **Usuário:** `postgres`
   - **Senha:** crie uma senha segura
4. Tipo de instância: `db.t3.micro`
5. Armazenamento: 20 GB (SSD General Purpose)
6. **Acesso público:** Habilitado (Sim)
7. **Nome do banco de dados inicial:** `pnad_covid`
8. Clique em **Criar banco de dados**

##### **2. Liberar o IP na VPC / Grupo de Segurança (Security Group)**

1. Vá para **EC2 > Grupos de Segurança**
2. Encontre o grupo associado à instância RDS
3. Clique em **Editar regras de entrada**
4. Adicione uma nova regra:
   - Tipo: `PostgreSQL`
   - Porta: `5432`
   - Origem: `Seu IP` (ou `0.0.0.0/0` temporariamente para teste – cuidado com isso em produção)
5. Salve as alterações.

✅ Agora o acesso externo ao banco estará liberado para seu IP

**Obs.: Essa configuração será feita apenas uma vez no VPC Default**

### **Códigos**

#### **Importar bibliotecas**

In [1]:
# Importar biblioteca completa
import requests
import zipfile
import io
import pandas as pd
import os
import boto3
import sys
import time 
import csv

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from botocore.exceptions import BotoCoreError, ClientError


#### **Funções (DEF)**

In [2]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("-", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)
        sys.exit()


In [3]:
# Calcular o tempo de execução
def calcular_tempo_execucao(tempo_inicio, tempo_final):
    """
    Calcula e exibe o tempo total de execução de um processo.

    Parâmetros:
    tempo_inicio (float): O tempo de início capturado com time.time()
    tempo_final (float): O tempo final capturado com time.time()

    A função não retorna nada, apenas imprime o resultado formatado.
    """
    # Calcula a diferença total em segundos
    tempo_total_segundos = tempo_final - tempo_inicio

    # Converte o total de segundos para minutos e segundos
    minutos, segundos = divmod(tempo_total_segundos, 60)

    # Imprime o resultado
    print(f"\nTempo total de execução do código:")
    print(f"{int(minutos)} minutos e {int(segundos)} segundos")

#### **Variaveis**

In [4]:
# Nome do bucket S3 e subpastas
s3_bucket = 'tech-challenge-fase3'
s3_subpasta_bronze = 'bronze'
s3_subpasta_silver = 'silver'
s3_subpasta_gold = 'gold'
pnad_consolidado = 'pnad_consolidado.parquet'

bronze_prefix = f"s3://{s3_bucket}/{s3_subpasta_bronze}/" 
silver_prefix = f's3://{s3_bucket}/{s3_subpasta_silver}/'
gold_prefix = f's3://{s3_bucket}/{s3_subpasta_gold}/'

# Nome arquivo .parquet
nome_arquivo_gold = 'pnad_final_tratado.parquet'

# Caminho de saida e entrada camada silver
caminho_saida_silver = silver_prefix + 'pnad_consolidado_enriquecido.parquet'
caminho_entrada_silver = silver_prefix + 'pnad_consolidado_enriquecido.parquet'
caminho_saida_gold = gold_prefix + 'pnad_final_tratado.parquet'
caminho_completo_gold = gold_prefix + nome_arquivo_gold

# Caminho do Github com dados do PNAD
api_url = 'https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados'

# Caminho do Github com dados do código IBGE UF
link_codigo_uf = 'https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv'

# Nome da tabela no PostgreSQL
nome_tabela = 'questionario_pnad_covid'
tabela_origem = nome_tabela

# Quantidade do chunksize
chunksize = 100000

# Carregar novamente a tabela 
carregar_tabela = 's'

# Role para usar no AWS Glue
role_arn = 'arn:aws:iam::430854566059:role/LabRole'

# Nome do banco e crawler do AWS Glue
db_name = 'pnad_db'
crawler_name = 'pnad_gold_crawler'
TablePrefix = 'pnad_'

#### **Configuração AWS e Banco de Dados**

In [5]:
# Carregar as credencias do .env
load_dotenv()

# Configuração storage_options
storage_options = {
    "key": os.getenv('AWS_ACCESS_KEY_ID'),
    "secret": os.getenv('AWS_SECRET_ACCESS_KEY'),
    "token": os.getenv('AWS_SESSION_TOKEN')
}

# Configuração S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
    region_name=os.getenv('AWS_REGION')
)

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER_PNAD")
senha_pg = os.getenv("POSTGRES_PASSWORD_PNAD")
host_pg = os.getenv("POSTGRES_HOST_PNAD")
porta_pg = os.getenv("POSTGRES_PORT_PNAD")
banco_pg = os.getenv("POSTGRES_DB_PNAD")

# Configuração Glue
glue_client = boto3.client('glue', region_name=os.getenv('region'))

#### **Validar Conexões e criar engine banco de dados**

In [6]:
# Validar conexão com a AWS através do .env
load_dotenv()

try:
    sts_client = boto3.client(
        'sts',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
        region_name=os.getenv('AWS_REGION')
    )
    
    identity = sts_client.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)

✅ Conectado à conta

UserId: AROAWIUHO6CVTPL5SPXB7:user4308167=ricardviana1@gmail.com
Account: 430854566059
Arn: arn:aws:sts::430854566059:assumed-role/voclabs/user4308167=ricardviana1@gmail.com


In [7]:
# Criar engine com banco 
engine = create_engine(f"postgresql+psycopg2://{usuario_pg}:{senha_pg}@{host_pg}:{porta_pg}/{banco_pg}")

# Testar a conexão
test_connection(engine)

✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
📄 Tabelas no banco:
- questionario_pnad_covid


#### **ETL**

In [8]:
# Registrar o tempo inicio do código
variavel_tempo_inicio = time.time()

In [9]:
# Criar tabela + Carregar dados no PostgreSQL

# Criar da tabela através de um pré-scan
if carregar_tabela.lower() == 'n':
    print(f'Etapa de carregar os dados do Github para o PostgreSQL não realizada pois a variavel carregar_tabela é `n`')

else:

    # Criar a tabela
    print("Iniciando Pré-Scan para definir estrutura unificada")

    try:
        response = requests.get(api_url) # Fazer requisição HTTP
        response.raise_for_status() # Verificar se a requisição foi bem feita
        files = response.json() # Transformar em uma lista de dicionario
        colunas_unicas = set() # Criar um conjuto vazio

        # Iterar sobre cada item no diretório do GitHub
        for file_info in files:

            if file_info['name'].endswith('.zip'):

                print(f"Analisando cabeçalho de: {file_info['name']}")

                zip_url = file_info['download_url']
                r_zip = requests.get(zip_url) # Fazer o download do arquivo
                zip_content = io.BytesIO(r_zip.content) # Arquivo temporario salvo em memoria

                with zipfile.ZipFile(zip_content) as z: # Abrir o arquivo .zip
                    csv_filename = next(f for f in z.namelist() if f.endswith('.csv'))

                    with z.open(csv_filename) as csv_file: # Abrir o arquivo .csv
                        df_header = pd.read_csv(csv_file, nrows=1) # Lê apenas as primeiras linhas para pegar o cabeçalho rapidamente
                        colunas_unicas.update(df_header.columns.tolist())

        lista_final_colunas = sorted(list(colunas_unicas))
        print(f"\n✅ Estrutura unificada definida com {len(lista_final_colunas)} colunas")

        # Cria um DataFrame vazio com a estrutura completa e unificada
        df_schema_unificado = pd.DataFrame(columns=lista_final_colunas)

        # Cria a tabela no PostgreSQL com esta estrutura completa
        print(f"\nCriando a tabela '{nome_tabela}' com a estrutura completa")
        df_schema_unificado.to_sql(nome_tabela, engine, if_exists='replace', index=False)
        print("✅ Tabela criada com sucesso")

    except Exception as e:
        print(f"❌ Erro durante o pré-scan ou criação da tabela: {e}")
        raise

    # Carregar os dados via Copy
    print(f"\nIniciando carga de dados para a tabela '{nome_tabela}'")
    
    try:
        with engine.connect() as connection: # Abrir conexão 
            raw_conn = connection.connection

            with raw_conn.cursor() as cursor: # Criar o cursor para executar os códigos

                # Iterar sobre cada item no diretório do GitHub
                for file_info in files:

                    if file_info['name'].endswith('.zip'):
                        
                        zip_name = file_info['name']
                        print(f"\nProcessando arquivo: {zip_name}")

                        zip_url = file_info['download_url']
                        r_zip = requests.get(zip_url) # Fazer o download do arquivo
                        zip_content = io.BytesIO(r_zip.content) # Arquivo temporario salvo em memoria

                        with zipfile.ZipFile(zip_content) as z: # Abrir o arquivo .zip
                            csv_filename = next(f for f in z.namelist() if f.endswith('.csv'))

                            with z.open(csv_filename) as csv_file: # Abrir o arquivo .csv
                                
                                # Carregar o CSV por pedaços (chunks)
                                chunk_iterator = pd.read_csv(csv_file, chunksize=chunksize, low_memory=False)
                                
                                # Iterar sobre cada pedaço e carregar para o banco de dados
                                for i, chunk in enumerate(chunk_iterator):
                                    
                                    buffer = io.StringIO()

                                    # Garantir que o chunk tenha todas as colunas da tabela, preenchendo as faltantes com None
                                    chunk_reindexed = chunk.reindex(columns=lista_final_colunas)
                                    chunk_reindexed.to_csv(buffer, index=False, header=False, quoting=csv.QUOTE_MINIMAL)
                                    buffer.seek(0)
                                    
                                    print(f"Carregando chunk {i+1} via COPY")
                                    sql_copy_command = f"COPY {nome_tabela} FROM STDIN WITH (FORMAT CSV, HEADER FALSE)"
                                    cursor.copy_expert(sql_copy_command, buffer)
                
                raw_conn.commit()
                print(f"\n✅ Carga de dados concluída com sucesso!")

    except Exception as e:
        print(f"❌ Erro durante a carga dos dados: {e}")
        if 'raw_conn' in locals() and raw_conn and not raw_conn.closed:
            raw_conn.rollback()
        raise

    # Inserir a coluna de data e hora + usuario
    print(f"\nAdicionando colunas de metadados à tabela '{nome_tabela}'")

    try:
        with engine.connect() as connection:
            
            # Comando para adicionar a coluna de data/hora da carga
            # IF NOT EXISTS: Garante que o script não dará erro se a coluna já existir
            # DEFAULT NOW(): Diz ao PostgreSQL para preencher este campo automaticamente com a data e hora atuais
            sql_add_timestamp = text(f"""
                ALTER TABLE {nome_tabela}
                ADD COLUMN IF NOT EXISTS data_hora_carga TIMESTAMPTZ DEFAULT NOW();
            """)
            
            # Comando para adicionar a coluna do usuário que fez a carga
            # DEFAULT CURRENT_USER: Preenche automaticamente com o nome do usuário do banco conectado.
            sql_add_user = text(f"""
                ALTER TABLE {nome_tabela}
                ADD COLUMN IF NOT EXISTS usuario_carga VARCHAR(255) DEFAULT CURRENT_USER;
            """)

            # Executa os comandos
            connection.execute(sql_add_timestamp)
            connection.execute(sql_add_user)
            
            # Efetiva as alterações na estrutura da tabela
            connection.commit() 

            print("✅ Colunas 'data_hora_carga' e 'usuario_carga' adicionadas com sucesso")

    except Exception as e:
        print(f"❌ Erro ao adicionar colunas de metadados: {e}")
        raise

Iniciando Pré-Scan para definir estrutura unificada
Analisando cabeçalho de: PNAD_COVID_052020.zip
Analisando cabeçalho de: PNAD_COVID_062020.zip
Analisando cabeçalho de: PNAD_COVID_072020.zip
Analisando cabeçalho de: PNAD_COVID_082020.zip
Analisando cabeçalho de: PNAD_COVID_092020.zip
Analisando cabeçalho de: PNAD_COVID_102020.zip
Analisando cabeçalho de: PNAD_COVID_112020.zip

✅ Estrutura unificada definida com 148 colunas

Criando a tabela 'questionario_pnad_covid' com a estrutura completa
✅ Tabela criada com sucesso

Iniciando carga de dados para a tabela 'questionario_pnad_covid'

Processando arquivo: PNAD_COVID_052020.zip
Carregando chunk 1 via COPY
Carregando chunk 2 via COPY
Carregando chunk 3 via COPY
Carregando chunk 4 via COPY

Processando arquivo: PNAD_COVID_062020.zip
Carregando chunk 1 via COPY
Carregando chunk 2 via COPY
Carregando chunk 3 via COPY
Carregando chunk 4 via COPY

Processando arquivo: PNAD_COVID_072020.zip
Carregando chunk 1 via COPY
Carregando chunk 2 via C

In [10]:
# Criar Bucket e subpastas (camadas medalhões)

# Lista com o nome das subpastas
pastas_a_criar = [s3_subpasta_bronze, s3_subpasta_silver, s3_subpasta_gold]

# Armazenar a região da conexão com a AWS
aws_region = s3_client.meta.region_name

print(f"\nIniciando Processo de Validação e Criação no S3")

print(f"\nValidando o bucket '{s3_bucket}'")
bucket_pronto = False

# Validar se o Bucket esta criado
try:
    s3_client.head_bucket(Bucket=s3_bucket)
    print(f"➡️  Bucket '{s3_bucket}' já existe")
    bucket_pronto = True

except ClientError as e:
    
    if e.response['Error']['Code'] == '404':
        print(f"Bucket '{s3_bucket}' não encontrado. Tentando criar")

        try:
            if aws_region == "us-east-1":
                s3_client.create_bucket(Bucket=s3_bucket)

            else:
                location = {'LocationConstraint': aws_region}
                s3_client.create_bucket(
                    Bucket=s3_bucket,
                    CreateBucketConfiguration=location
                )
            print(f"✅ Bucket '{s3_bucket}' criado com sucesso!")
            bucket_pronto = True

        except Exception as create_e:
            print(f"❌ Falha ao tentar criar o bucket: {create_e}")

    else:
        print(f"❌ Erro de permissão ou outro problema ao verificar o bucket: {e}")

# Validar se as subpastas estão criadas
if bucket_pronto:
    print(f"\nValidando as subpastas no bucket '{s3_bucket}'")

    for nome_pasta in pastas_a_criar:
        chave_pasta = nome_pasta if nome_pasta.endswith('/') else nome_pasta + '/'
        
        try:
            s3_client.head_object(Bucket=s3_bucket, Key=chave_pasta)
            print(f"➡️ Pasta '{chave_pasta}' já existe")

        except ClientError as e:

            if e.response['Error']['Code'] == '404':
                try:
                    s3_client.put_object(
                        Bucket=s3_bucket,
                        Key=chave_pasta,
                        Body=''
                    )
                    print(f"✅ Pasta '{chave_pasta}' criada com sucesso")

                except Exception as create_e:
                    print(f"❌ Falha ao TENTAR CRIAR a pasta '{chave_pasta}': {create_e}")

            else:
                print(f"❌ Erro ao verificar a pasta '{chave_pasta}': {e}")
else:
    print("\nCriação das pastas abortada, pois houve um problema com o bucket")

print("\nProcesso Finalizado")


Iniciando Processo de Validação e Criação no S3

Validando o bucket 'tech-challenge-fase3'
➡️  Bucket 'tech-challenge-fase3' já existe

Validando as subpastas no bucket 'tech-challenge-fase3'
➡️ Pasta 'bronze/' já existe
➡️ Pasta 'silver/' já existe
➡️ Pasta 'gold/' já existe

Processo Finalizado


In [11]:
# Carregar os dados do PostgreSQL para o S3 

if carregar_tabela.lower() == 'n':
    print(f'Etapa de carregar os dados do PostgreSQL para o S3 não realizada pois a variavel carregar_tabela é `n`')

else:
    print(f"Iniciando extração da tabela '{tabela_origem}'")

    try:
        print("\nCalculando o total de registros a serem processados")
        with engine.connect() as connection:
            total_de_linhas = connection.execute(text(f"SELECT COUNT(*) FROM {tabela_origem}")).scalar()
        print(f"Total de registros encontrado: {total_de_linhas}")

        sql_query = f"SELECT * FROM {tabela_origem}"
        print(f"\nAguarde, executando o comando {sql_query} e isso pode levar um determinado tempo")
        chunk_iterator = pd.read_sql_query(sql_query, engine, chunksize=chunksize)
        
        total_registros_processados = 0

        print(f"\nProcessando e salvando dados na Camada Bronze em '{bronze_prefix}'")
        for i, chunk_df in enumerate(chunk_iterator):
            
            num_linhas_chunk = len(chunk_df)
            total_registros_processados += num_linhas_chunk
            
            percentual_concluido = (total_registros_processados / total_de_linhas) * 100
            print(f"\nProcessando chunk {i+1} ({percentual_concluido:.0f}%): {num_linhas_chunk} linhas")
            
            caminho_arquivo_parquet = f"{bronze_prefix}part-{i:04d}.parquet"
            
            chunk_df.to_parquet(
                caminho_arquivo_parquet,
                engine='pyarrow',
                index=False,
                storage_options=storage_options
            )
            print(f"✅ Chunk {i+1} salvo em '{caminho_arquivo_parquet}'")

        print(f"\n✅ Extração concluída! Total de {total_registros_processados} registros salvos em múltiplos arquivos Parquet")

    except Exception as e:
        print(f"❌ Erro durante a extração em chunks: {e}")
        raise

Iniciando extração da tabela 'questionario_pnad_covid'

Calculando o total de registros a serem processados
Total de registros encontrado: 2650459

Aguarde, executando o comando SELECT * FROM questionario_pnad_covid e isso pode levar um determinado tempo

Processando e salvando dados na Camada Bronze em 's3://tech-challenge-fase3/bronze/'

Processando chunk 1 (4%): 100000 linhas
✅ Chunk 1 salvo em 's3://tech-challenge-fase3/bronze/part-0000.parquet'

Processando chunk 2 (8%): 100000 linhas
✅ Chunk 2 salvo em 's3://tech-challenge-fase3/bronze/part-0001.parquet'

Processando chunk 3 (11%): 100000 linhas
✅ Chunk 3 salvo em 's3://tech-challenge-fase3/bronze/part-0002.parquet'

Processando chunk 4 (15%): 100000 linhas
✅ Chunk 4 salvo em 's3://tech-challenge-fase3/bronze/part-0003.parquet'

Processando chunk 5 (19%): 100000 linhas
✅ Chunk 5 salvo em 's3://tech-challenge-fase3/bronze/part-0004.parquet'

Processando chunk 6 (23%): 100000 linhas
✅ Chunk 6 salvo em 's3://tech-challenge-fase3/bro

In [12]:
# Ler arquivos CSV Codigo IBGE e gerar Data Frame
print(f"\nLendo tabela com os códigos do IBGE de: {link_codigo_uf}")
df_uf = pd.read_csv(link_codigo_uf, sep=",")
print("Data Frame criado com sucesso")


Lendo tabela com os códigos do IBGE de: https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv
Data Frame criado com sucesso


In [13]:
# Carregar dados para camada silver

# Ler arquivos CSV PNAD COVID do S3, consolidar e gerar Data Frame
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=f'{s3_subpasta_bronze}/') # Listar todos os objetos (arquivos) dentro do S3
bronze_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.parquet')] # Pegar apenas arquivos .csv
lista_de_dataframes = []

print("Lendo arquivos da camada Bronze no S3")

# Iterar sobre os arquivos
for file_key in bronze_files:

    file_path = f"s3://{s3_bucket}/{file_key}"
    print(f"Lendo {file_path}")
    df_temp = pd.read_parquet(file_path, storage_options=storage_options)
    lista_de_dataframes.append(df_temp)

# Consolidar os data frame
df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)
print(f"\nConsolidação concluída e {len(df_consolidado)} linhas lidas da camada Bronze")

# Converter a coluna UF e Código para objetct
df_consolidado["UF"] = df_consolidado["UF"].astype(str)
df_uf["Código"] = df_uf["Código"].astype(str)

# Relacionar os Data Frame
print("\nIniciando enriquecimento dos dados com merge com o Data Frame df_uf")
df_silver = pd.merge(
    df_consolidado,
    df_uf,
    how='left',
    left_on='UF', 
    right_on='Código'
)

# Remover colunas não necessarias e ajustar o nome das colunas
df_silver = df_silver.drop(columns=["Código"])
df_silver = df_silver.rename(columns={"UF_x": "UF", "UF_y": "UF_Nome", "Região": "Regiao"})
print("Enriquecimento concluído")

colunas_para_tratar = ["A006A", "A006B"]

# Iterar sobre cada coluna da lista
for coluna in colunas_para_tratar:
    
    # Passo 1: Converter a coluna para um tipo numérico (float).
    # 'errors='coerce'' é muito útil: se ele encontrar um texto que não consegue converter
    # (ex: "não aplicável"), ele transforma esse valor em NaN (nulo) em vez de dar erro.
    df_silver[coluna] = pd.to_numeric(df_silver[coluna], errors='coerce')
    
    # Passo 2: Agora que tudo é numérico (ou NaN), podemos preencher os nulos com 0.
    df_silver[coluna] = df_silver[coluna].fillna(0)
    
    # Passo 3: Finalmente, com a coluna limpa e sem decimais indesejados, convertemos para inteiro.
    df_silver[coluna] = df_silver[coluna].astype(int)

# Ver o resultado final
print("\nData Frame com os primeiros registros")
display(df_silver.head(5))

print("Data Frame com os ultimos registros")
display(df_silver.tail(5))

Lendo arquivos da camada Bronze no S3
Lendo s3://tech-challenge-fase3/bronze/part-0000.parquet
Lendo s3://tech-challenge-fase3/bronze/part-0001.parquet
Lendo s3://tech-challenge-fase3/bronze/part-0002.parquet
Lendo s3://tech-challenge-fase3/bronze/part-0003.parquet
Lendo s3://tech-challenge-fase3/bronze/part-0004.parquet
Lendo s3://tech-challenge-fase3/bronze/part-0005.parquet


KeyboardInterrupt: 

In [ ]:
# Salvar Data Frame df_silver na camada silver
print(f"\nSalvando dados da camada Silver em: {caminho_saida_silver}")

df_silver.to_parquet(
    caminho_saida_silver,
    index=False,
    storage_options=storage_options
)

print("✅ Sucesso! Camada Silver criada e salva no S3")

In [ ]:
# Tratar os dados para a camada Gold
print(f"Lendo dados da camada Silver de: {caminho_entrada_silver}")

try:
    df_silver = pd.read_parquet(caminho_entrada_silver, storage_options=storage_options)
    print(f"Leitura concluída onde o dataFrame contem {len(df_silver)} linhas e {len(df_silver.columns)} colunas")

except Exception as e:
    print(f"❌ Erro ao ler o arquivo da camada Silver. Verifique o caminho e as permissões. Detalhes: {e}")
    sys.exit()

# Copiar e padronizar colunas para minúsculas
df_estruturado = df_silver.copy()
df_estruturado.columns = df_estruturado.columns.str.lower()

# Colunas fixas (com descrição)
colunas_fixas = [
    'ano',       # Ano
    'v1013',     # Mês do Ano
    'v1012',     # Semana do Mês
    'uf',        # Sigla da Unidade da Federação
    'capital',   # Capital do Estado
    'rm_ride',   # Região Metropolitana e Região Administrativa Integrada de Desenvolvimento
    'uf_nome',   # Nome do UF
    'sigla',     # Sigla do UF
    'regiao',    # Região
]

# Colunas desejadas (com descrição)
colunas_desejadas = [
    'a002',      # Idade
    'a003',      # Sexo
    'a004',      # Raça ou Cor
    'a006b',     # Você está tendo aulas presenciais?
    'b008',      # O(A) Sr(a) fez algum teste para saber se estava infectado(a) pelo coronavírus?
    'b009a',     # Fez o exame coletado com cotonete na boca e/ou nariz (SWAB)?
    'b009c',     # Fez o exame de coleta de sangue através de furo no dedo?
    'b009e',     # Fez o exame de coleta de sangue através da veia do braço?
    'a005',      # Escolaridade
    'a006',      # Frequenta escola
    'a006a',     # A escola/faculdade que frequenta é pública ou privada?
    'b0011',     # Na semana passada teve febre?
    'b0012',     # Na semana passada teve tosse?
    'b0013',     # Na semana passada teve dor de garganta?
    'b0014',     # Na semana passada teve dificuldade para respirar?
    'b0015',     # Na semana passada teve dor de cabeça?
    'b0016',     # Na semana passada teve dor no peito?
    'b0017',     # Na semana passada teve náusea?
    'b0018',     # Na semana passada teve nariz entupido ou escorrendo?
    'b0019',     # Na semana passada teve fadiga?
    'b00110',    # Na semana passada teve dor nos olhos?
    'b00111',    # Na semana passada teve perda de cheiro ou sabor?
    'b00112',    # Na semana passada teve dor muscular?
    'b00113',    # Na semana passada teve diarreia?
]

# Unir listas mantendo a ordem e sem duplicar
todas_colunas = list(dict.fromkeys([*colunas_fixas, *colunas_desejadas]))

# Manter apenas as colunas que existem no DataFrame após padronização
colunas_existentes = [c for c in todas_colunas if c in df_estruturado.columns]

# Filtro com os 3 últimos valores de v1013
df_estruturado["v1013"] = df_estruturado["v1013"].astype(int)

if 'v1013' in df_estruturado.columns:
    # Coletar valores únicos (excluindo NaN), ordenar e pegar os 3 últimos
    ultimos_3 = sorted(pd.unique(df_estruturado['v1013'].dropna()))[-3:]

    # Filtrar registros pertencentes aos 3 últimos meses e selecionar colunas
    estrutura_final = (
        df_estruturado[df_estruturado['v1013'].isin(ultimos_3)][colunas_existentes]
        .reset_index(drop=True)
    )
else:
    # Se não houver v1013, apenas seleciona as colunas existentes
    estrutura_final = df_estruturado[colunas_existentes].reset_index(drop=True)

# Avisos sobre colunas ausentes
faltantes = [c for c in todas_colunas if c not in df_estruturado.columns]
if faltantes:
    print(f"Atenção: estas colunas não foram encontradas e ficaram de fora: {faltantes}")

# Converter colunas para o formato correto
colunas_para_converter = [
    'a002', 'a003', 'a004', 'a005', 'a006', 'a006a', 'a006b'
    'b0011', 'b0012', 'b0013', 'b0014', 'b0015', 'b0016', 'b0017',
    'b0018', 'b0019', 'b00110', 'b00111', 'b00112', 'b00113',
    'b008', 'b009a', 'b009c', 'b009e'
]

# Itera sobre a lista e converte cada coluna
for col in colunas_para_converter:
    # 'errors='coerce'' é um parâmetro de segurança: se algum valor não puder
    # ser convertido para número, ele se tornará NaN (nulo), evitando que o código quebre
    if col in estrutura_final.columns:
        estrutura_final[col] = pd.to_numeric(estrutura_final[col], errors='coerce')

# Tratar o campo de sintomas
estrutura_final["febre"] = (estrutura_final['b0011'] == 1).astype(int)
estrutura_final["tosse"] = (estrutura_final["b0012"] == 1).astype(int)
estrutura_final["dor_garganta"] = (estrutura_final["b0013"] == 1).astype(int)
estrutura_final["dificuldade_respirar"] = (estrutura_final["b0014"] == 1).astype(int)
estrutura_final["dor_cabeca"] = (estrutura_final["b0015"] == 1).astype(int)
estrutura_final["dor_peito"] = (estrutura_final["b0016"] == 1).astype(int)
estrutura_final["nausea"] = (estrutura_final["b0017"] == 1).astype(int)
estrutura_final["nariz_entupido"] = (estrutura_final["b0018"] == 1).astype(int)
estrutura_final["fadiga"] = (estrutura_final["b0019"] == 1).astype(int)
estrutura_final["dor_olhos"] = (estrutura_final["b00110"] == 1).astype(int)
estrutura_final["perda_olfato"] = (estrutura_final["b00111"] == 1).astype(int)
estrutura_final["dor_muscular"] = (estrutura_final["b00112"] == 1).astype(int)
estrutura_final["diarreia"] = (estrutura_final["b00113"] == 1).astype(int)
estrutura_final["soma_sintomas"] = estrutura_final["febre"] + estrutura_final["tosse"] + estrutura_final["dor_garganta"] + estrutura_final["dificuldade_respirar"] + estrutura_final["dor_cabeca"] + estrutura_final["dor_peito"] + estrutura_final["nausea"] + estrutura_final["nariz_entupido"] + estrutura_final["fadiga"] + estrutura_final["dor_olhos"] + estrutura_final["perda_olfato"] + estrutura_final["dor_muscular"] + estrutura_final["diarreia"]

# Tratar o campo de sexo e raça
estrutura_final['sexo'] = estrutura_final['a003'].apply(lambda x: 'homem' if x == 1 else ('mulher' if x == 2 else None))

raca_mapping = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indigena',
    9: 'Ignorado'
}

estrutura_final['raca'] = estrutura_final['a004'].map(raca_mapping)

# Tratar o campo de teste
estrutura_final["fez_teste"] = (estrutura_final["b008"] == 1).astype(int)
estrutura_final["teste_swab"] = (estrutura_final["b009a"] == 1).astype(int)
estrutura_final["teste_furo_dedo"] = (estrutura_final["b009c"] == 1).astype(int)
estrutura_final["teste_exame_sangue"] = (estrutura_final["b009e"] == 1).astype(int)

# Tratar o campo de escola
escolaridade_map = {
    1: 'Sem instrução',
    2: 'Fundamental incompleto',
    3: 'Fundamental completa',
    4: 'Médio incompleto',
    5: 'Médio completo',
    6: 'Superior incompleto',
    7: 'Superior completo',
    8: 'Pós-graduação, mestrado ou doutorado',
    9: 'Ignorado'
}

tipo_escola_map = {
    1: 'Publica',
    2: 'Privada'
}

aulas_presenciais_map = {
    1: 'Sim, normalmente',
    2: 'Sim, mas apenas parcialmente',
    3: 'Não, e meu normalmente é presencial/semipresencial',
    4: 'Não, meu curso é online'
}

estrutura_final['escolaridade'] = estrutura_final['a005'].map(escolaridade_map)
estrutura_final['tipo_escola'] = estrutura_final['a006a'].map(tipo_escola_map)
estrutura_final['aulas_presenciais'] = estrutura_final['a006b'].map(aulas_presenciais_map)
estrutura_final["frequenta_escola"] = (estrutura_final["a006"] == 1).astype(int)

# Ver o resultado final
print("\nData Frame com os primeiros registros")
display(estrutura_final.head(5))

print("Data Frame com os ultimos registros")
display(estrutura_final.tail(5))


In [ ]:
# Validar a camada silver vs a nova camada gold
linhas, colunas = df_silver.shape
print(f"O DataFrame df_silver tem {linhas} linhas")
print(f"O DataFrame df_silver tem {colunas} colunas")

linhas, colunas = estrutura_final.shape
print(f"\nO DataFrame estrutura_final tem {linhas} linhas")
print(f"O DataFrame estrutura_final tem {colunas} colunas")

In [ ]:
# Carregar os dados para a Gold
print(f"\nSalvando dados da camada Gold em: {caminho_saida_gold}")

# Salvar o data frame como .parquet 
estrutura_final.to_parquet(
    caminho_saida_gold,
    index=False,
    storage_options=storage_options # Parametro necessario para conseguir conectar o Pandas ao AWS
)

print(f"✅ Sucesso! Camada Gold criada com {len(estrutura_final)} linhas e salva no S3.")

In [ ]:
# Utilizar o Glue para catalogar a camada gold usar o Athena

# Criar o banco de dados se não existir
try:
    glue_client.get_database(Name=db_name)
    print(f"➡️ Banco de dados '{db_name}' já existe")
    
except glue_client.exceptions.EntityNotFoundException:
    print(f"Criando banco de dados '{db_name}'")
    glue_client.create_database(DatabaseInput={'Name': db_name})
    print(f"✅ Banco de dados '{db_name}' criado")

# Criar o Crawler se não existir
try:
    glue_client.get_crawler(Name=crawler_name)
    print(f"\n➡️ Crawler '{crawler_name}' já existe")

except glue_client.exceptions.EntityNotFoundException:
    print(f"Criando crawler '{crawler_name}'")
    glue_client.create_crawler(
        Name=crawler_name,
        Role=role_arn,
        DatabaseName=db_name,
        Targets={
            'S3Targets': [
                {
                    'Path': gold_prefix,
                }
            ]
        },
        TablePrefix = TablePrefix
    )
    print(f"✅ Crawler '{crawler_name}' criado")

# Executar o Crawler
print(f"\nIniciando a execução do crawler '{crawler_name}'")
glue_client.start_crawler(Name=crawler_name)

# Monitorar a execução do crawler
while True:
    try:
        crawler_status = glue_client.get_crawler(Name=crawler_name)
        state = crawler_status['Crawler']['State']
        last_crawl_info = crawler_status['Crawler'].get('LastCrawl', {})
        status_detail = last_crawl_info.get('Status', 'N/A')
        
        print(f"Status do Crawler: {state} | Detalhe da Última Execução: {status_detail}")

        if state == 'READY':
            if status_detail == 'SUCCEEDED':
                print("\n✅ Crawler finalizou a execução com sucesso!")
            elif status_detail in ['FAILED', 'CANCELLED']:
                print(f"\n❌ A última execução do crawler falhou ou foi cancelada. Detalhe: {last_crawl_info.get('ErrorMessage', 'Sem detalhes')}")
            else:
                 print("✅ Crawler está pronto para a próxima execução.")
            break # Sair do loop em qualquer cenário onde o crawler está 'READY'

        elif state == 'FAILED':
            print(f"❌ Erro crítico no crawler. Detalhe: {crawler_status['Crawler'].get('LastCrawl', {}).get('ErrorMessage', 'Sem detalhes')}")
            break

        # Se estiver em RUNNING ou STOPPING, continua esperando
        time.sleep(30)

    except Exception as e:
        print(f"Ocorreu um erro ao monitorar o crawler: {e}")
        break

In [ ]:
# Registrar o tempo final do código
variavel_tempo_final = time.time()

# Gerar o resultado
calcular_tempo_execucao(variavel_tempo_inicio, variavel_tempo_final)